# Tutorial 2, Part 1: Impact of uncentered measurements on functional connectivity

The standard correlation matrix assumes data is distributed according to a one-way multivariate model. This assumption is violated in spatio-temporal measurements that come from fMRI.

<!----# Introduction to Two-Way Models ---->
<!---- Consider two bi-variate time-series, representing BOLD signals at either two voxels or two regions of interest. ----->
<!---- $x_{1,t} = \mu_1 + e_{1,t}, \quad x_{2,t} = \mu_2 + e_{2,t}$ ---->

### Content

#### 1. [Simulate null functional connectivity using centered one-way multivariate normal](#one-way)
#### 2. [Simulate null functional connectivity using row-uncentered two-way multivariate normal](#two-way)
#### 3. [Histogram of correlation coefficients without centering two-way data](#biased-correlations)

Conclusion: Standard pairwise correlation intended for one-way measurements is a statistically biased estimator for uncentered two-way measurements

### Simulate one-way multivariate data. 
Default simulation uses `p=50` brain regions and `m=200` BOLD fMRI volumes or measurements with null correlation matrix `sigma`
<a id="one-way"></a>

In [3]:
% Simulate one-way multivariate data 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Dimensions or Features 
p = 50; 
% Measurements or Observations
m = 200; 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% Multivariate, i.i.d normal data
rcen_mean = @(mu)(repmat(mu,[m 1])); 
rcen_mvnsim = @(mu,sigma)(rcen_mean(mu) + randn(m,p)*sqrtm(sigma)); 
title = @(tstring)(title(tstring,'fontsize',16)); 

mu = zeros(1,p); 
sigma = eye(p); 
X = rcen_mvnsim(mu,sigma); 
figure;
subplot(1,2,1); 
imagesc(rcen_mean(mu)); colorbar;
title('Row-centered, Mean')
subplot(1,2,2); 
Shat_centered = corr(X); 
imagesc(Shat_centered); colormap(summer);  colorbar; axis image equal;
title(sprintf('Sample Correlation. \n  Centered mean m=%d, p=%d',m,p)); set(gca,'fontsize',18)
figure;
hist_norm = 1;
hist(triu(Shat_centered,1),100,hist_norm);ylim([0 hist_norm*1.1]); xlim([-.5 .5]); 
title('Histogram of off-diagonal correlations')

### Simulate null functional connectivity using row uncentered two-way model
<a id="two-way"></a>
Default simuluation now adds a random offset for each measurement. This can be modified by setting `rsignal` to some non-zero value. 

In [4]:
% Simulate row uncentered two-way multivariate data 
% Vary amount of the row offset by changing signal. Default signal = 1; 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
rsignal = 2; csignal = 0;  rand('seed',0); 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% %Constant offset for each feature
% row_offset = ones(m,1)*.25*sqrt(signal);
%
% Random offset for each observation
row_offset = rsignal + randn(m,1)*.25*sqrt(rsignal);     
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Random offset for each feature. Default 0. 
mu = csignal + randn(1,p)*.25*sqrt(csignal);     
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

rncen_mean = @(mu)(repmat(mu,[m 1])+repmat(row_offset,[1 p])); 
rncen_mvnsim = @(mu,sigma)(rncen_mean(mu) + randn(m,p)*sqrtm(sigma)); 

### Examine the uncentered data
<a id="uncentered-data"></a>

In [8]:
Xnc = rncen_mvnsim(mu,sigma); 
disp('Center the features (columns or brain regions or voxels)')
Xnc = bsxfun(@minus,Xnc,mean(Xnc));
mu_c_hat = mean(Xnc);
mu_r_hat = mean(Xnc');
disp('.....')
disp('Mean signal corresponding to Regions.  Region 1, ...., Region 5')
mu_c_hat(1:5)
disp('Mean "brain wide" signal at each measurement due to measurement error. Observation 1, ...., Observation 5')
mu_r_hat(1:5)

Center the features (columns or brain regions or voxels)


.....


Mean signal corresponding to Regions.  Region 1, ...., Region 5


ans =


  -8.2601e-16  -4.7296e-16  -1.1458e-15  -1.0514e-15  -9.7478e-16


Mean "brain wide" signal at each measurement due to measurement error. Observation 1, ...., Observation 5


ans =


  -0.154242   0.075572   0.353733  -0.690877   0.017067


### Histogram of correlation coefficients
<a id="biased-correlations"></a>

Though all population correlation coefficients are zero 0. Sample correlation coefficients are statistically biased due to uncentered measurements. 
Excercise: Try increasing the number of measurements $m$. 

In [10]:
Xnc = rncen_mvnsim(mu,sigma); 
figure;
subplot(1,2,1); 
imagesc(rncen_mean(mu)); colorbar; 
title('Row uncentered, Mean')
subplot(1,2,2); 
Shat_ncentered = corr(Xnc);  
imagesc(Shat_ncentered); colormap(summer);  colorbar; axis image equal;
title(sprintf('Sample Correlation. \n Uncentered mean m=%d, p=%d',m,p)); set(gca,'fontsize',18)
figure;
hist_norm = 1;
hist(triu(Shat_ncentered,1),100,hist_norm);ylim([0 hist_norm*1.1]); xlim([-.5 .5]); 
title('Histogram of off-diagonal correlations')